In [2]:
import pandas as pd

In [778]:
dataset = pd.read_csv('weather_results.csv')
pd.set_option('display.max_columns', None)

In [801]:
dataset.sample(1)

,ID,Sky,Temperature,Precipitation,Wind,Animal,Place,CSW,CSS,CSS_and,CSS_or,CSS_both,CSP,CSP_simple,CSP_know,CSP_gen,CSP_pizza,CSP_drizzle,CSP_shower,S1,S2,CSA,T5,Falcon
142,142,overcast,27,16,11,fish,city,There is no wind,There is smog,There is smog,There is smog,There is smog,There is rain,There is rain.,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"S, P, T, W","S, P, T, W","S, P, T","Tomorrow in the city S, P, T, W.",\nThe weather in the city tomorrow will be mostly sunny with a chance of showers in the afternoon.


## SKY

In [144]:
# SKY SIMPLE; 86x overcast(fully ok), 91x cloudy (fully ok), 217 clear (should be 106), 109x smog (should be 172), 97x mist (should be 145)
# If [VAR:place] is beach, there is mist. If [VAR:place] is city, there is smog.
dataset[(dataset.Place=='beach') & (~dataset['CSS'].str.contains('mist'))] # 97 ok, 48 did not get it - likes it clear sky
dataset[(dataset.Place=='city') & (~dataset['CSS'].str.contains('smog'))] # 109 ok, 63 - clear sky 
dataset[(dataset['Sky'].str.contains('clear')) & (dataset['Place']!='beach') & (dataset['Place']!='city')]
dataset[((dataset.Place=='beach') )]
print('accuracy:', (600-48-63)/600)


accuracy: 0.815


In [192]:
# SKY AND - should be: 56 mist but 15x clear; 172 smog - 98x not (63x clear, 35xcloudy); clear - should be 139 - ok;  cloudy should be 118 - but 47x mist; overcast should be 115 - but 56x mist
# If [VAR:place] is beach and [VAR:animal] is cat, there is mist. If [VAR:place] is city, there is smog.
len(dataset[(dataset.Sky=='overcast') & (dataset.Place!='city') & (~((dataset.Place=='beach') & (dataset.Animal=='cat'))) & ~(dataset.CSS_and.str.contains('overcast'))])
print('accuracy:', (600-15-98-47-56)/600))

accuracy: 0.64


In [800]:
len(dataset[((dataset.Place=='beach') | (dataset.Animal=='cat')) & (dataset.Place!='city') & (dataset.CSS_and.str.contains('overcast'))]) # 244, 144 mist, 87 clear, cloudy 13

0

In [388]:
# SKY OR - smog - should be 264 but 202 not (29x mist, 117 clear, 34 cloudy 22 overcast); mist - should be 145 but 48x clear´; clear -  70 ok; cloudy - 57 ok; overcast - 64 ok
# the thing is what happens if there is bird -> should be smog and there is beach -> shoud be mist! - seems like for this it always chooses mist! (29x)
# If [VAR:place] is beach, there is mist. If [VAR:place] is city or [VAR:animal] is bird, there is smog.
len(dataset[((dataset.Place=='city') | (dataset.Animal=='bird')) & (dataset.CSS_or.str.contains('mist'))])
len(dataset[(~((dataset.Place=='city') | (dataset.Animal=='bird'))) & (dataset.Place!='beach') & (dataset.Sky=='overcast') & (dataset.CSS_or.str.contains('overcast'))])
print('accuracy:', (600-202-48)/600, 'or for mist correct:', (600-202+29-48)/600)


accuracy: 0.5833333333333334 or for mist correct: 0.6316666666666667


In [408]:
len(dataset[((dataset.Place=='city') | (dataset.Animal=='bird') & (dataset.Place!='beach'))])  # no city only bird - no smog, 139 something
#40x both city and bird, 21x not chooses it - all for sky is clear; city no bird 42x clear 69x smog
len(dataset[((dataset.Place!='city') & (dataset.Animal=='bird')) & (dataset.CSS_or.str.contains('overcast'))] ) # it does not select smog when only bird!


22

In [526]:
# SKY BOTH - smog 311 but 237 (29 mist, 117 clear, 69 cloudy, 22 overcast); mist 56 but 15 clear (same); clear 85 ok ; cloudy 70 but 33 mist ; overcast 78 but 41x mist
# If [VAR:place] is beach and [VAR:animal] is cat, there is mist. If [VAR:place] is city or [VAR:animal] is bird, there is smog.
len(dataset[((dataset.Place=='beach') & (dataset.Animal=='cat')) & (dataset.CSS_both.str.contains('clear'))])
len(dataset[(~((dataset.Place=='city') | (dataset.Animal=='bird')) & ~((dataset.Place=='beach') & (dataset.Animal=='cat'))) & (dataset.Sky=='overcast') & (~dataset.CSS_both.str.contains('overcast'))])
print('accuracy:', (600-237+29-15-33-41)/600)

accuracy: 0.505


In [545]:
len(dataset[(dataset.Animal=='bird') & ~(dataset.Place=='city') & (dataset.CSS_both.str.contains('mist'))]) # when city 77 clear sky, 34 smog, 35 cloudy
# when only bird: 85 not smog and not clear; no smog, 54 clear, 34 cloudy, 22 overcast, 29 mist

29

## WIND

In [309]:
# WIND;  119 < 269 < 212 (144, 235, 221 instances); no wind - all 144 ok; wind - 9 strong - it is == 50 not <50); strong wind all 212 ok
# Rules: If [VAR:wind] is less than 18 there is no wind. If [VAR:wind] is more than 50 there is strong wind.

dataset[(dataset.Wind > 50) & (~dataset.CSW.str.contains('strong wind'))] 
dataset[(dataset.Wind < 18) & (dataset.CSW.str.contains('no wind'))] 
dataset[(dataset.Wind >= 18) & (dataset.Wind <= 50) & (~dataset.CSW.str.contains('is wind'))]

print('accuracy:', (600-9)/600)

accuracy: 0.985


## PRECIPITATION

In [332]:
# CSP simple, no precip - shoud be 358 but 62 snow (temp ok) and 1 rain (clear sky) + if therewould be and (precip&sky) it would be ok; 
# snow shoud be 59 all ok; rain shoud be 183 but 11 snow (temp 0 or 1 or 2)
# If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow. If [VAR:sky] is clear, there is no rain and no snow.
dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & (~dataset.CSP_simple.str.contains('no rain'))]
dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature<0) & (~dataset.CSP_simple.str.contains('is snow'))]
dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (~dataset.CSP_simple.str.contains('is rain'))]
print('accuracy:', (600-63-11)/600)

accuracy: 0.8766666666666667


In [785]:
# knowledge
# no precip - shoud be 358 (212ok) but 106 snow (temp ok) and 38 rain (clear sky)
#snow shoud be 59 all ok; rain shoud be 183 (119 ok) but 40 snow with max Temp 9 and 24x no rain with + temperatures and precipitation even to 19

dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & (~dataset.CSP_know.str.contains('no rain')) & (dataset.Sky=='clear')]
#dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature<0) & (~dataset.CSP_know.str.contains('is snow'))]
#dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (~dataset.CSP_know.str.contains('is rain'))]
print('accuracy:', (600-106-38-40-24)/600)

accuracy: 0.6533333333333333


In [460]:
# Pizza
# no precip - shoud be 358 (177 ok) but 98 snow (temp soso) and 83 rain (clear sky, temp ok), no pizza
#snow 59 all ok; rain shoud be 183 (153 ok) but 30 snow (max Temp 5); pizza 0

len(dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & (dataset.CSP_pizza.str.contains('is pizza'))])
len(dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature<0) & (~dataset.CSP_pizza.str.contains('is snow'))])
dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (dataset.CSP_pizza.str.contains('no rain'))]
print('accuracy:', (600-98-83-30)/600)

accuracy: 0.6483333333333333


In [473]:
# Drizzle
# no precip - shoud be 358 (177 ok) but 94 snow (temp soso) and 87 rain (clear sky, temp ok), no drizzle
#snow 59 all ok; rain shoud be 183 (161 ok) but 22 snow (max Temp 4); 1x no rain with precip 18; no dirzzle

len(dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & (dataset.CSP_drizzle.str.contains('is rain'))])
len(dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature<0) & (~dataset.CSP_drizzle.str.contains('is snow'))])
dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (dataset.CSP_drizzle.str.contains('no rain'))]
print('accuracy:', (600-94-87-21)/600)

accuracy: 0.6633333333333333


In [493]:
# Shower
# no precip - shoud be 358 (177 ok) but 92 snow (temp soso) and 78 rain (clear sky, temp ok), 11x shower!
#snow 59 all ok; rain shoud be 183 (156 ok) but 21 snow (max Temp 5); 5x shower!

len(dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & (dataset.CSP_shower.str.contains('is shower'))])
len(dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature<0) & (~dataset.CSP_shower.str.contains('is snow'))])
len(dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (dataset.CSP_shower.str.contains('no rain'))])
print('accuracy:', (600-92-78-11-21-5)/600)

accuracy: 0.655


In [518]:
# Generative
# no precip - shoud be 358 (343 ok) but 15 snow (temp ok - no precip)
#snow 59 all ok; rain shoud be 183 but 84 snow (max Temp 29)

len(dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & ~( (dataset.CSP_gen.str.contains('no snow', case=False)) | (dataset.CSP_gen.str.contains('no rain', case=False)) )])
dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature<0) & (~dataset.CSP_gen.str.contains('snow', case=False))]
dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (dataset.CSP_gen.str.contains('no rain', case=False))]
print('accuracy:', (600-15-84)/600)

accuracy: 0.835


In [781]:
# CSP complex
# If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow. If [VAR:sky] is clear, there is no rain and no snow. 
# If there is rain and the [VAR:place] is mountains, there is storm.
# snow - should be 59 - 13 wrong storm (only once got right for mountains), rain - should be 134 (120 right) 14x snow - around 0 but also 2x11 and 1x8)
# nothing - should be 358 - 11 wrong (8x snow temp ok, 3x storm - but is clear - and 95 mountains only 3 wrong), storm - 49 as should be.

# shoud be 358 but 62 snow (temp ok) and 1 rain (clear sky) + if therewould be and (precip&sky) it would be ok; 
# rain shoud be 183 but 11 snow (temp 0 or 1 or 2)

len(dataset[(dataset.Precipitation!=0) & (dataset.Sky!='clear') & (dataset.Temperature>=0) & (dataset.Place=='mountains') & (dataset.CSP.str.contains('is storm'))])
dataset[((dataset.Precipitation==0) | (dataset.Sky=='clear')) & (dataset.CSP.str.contains('is storm'))]
len(dataset[((dataset.Precipitation!=0) & (dataset.Sky!='clear')) & (dataset.Temperature>=0) & (dataset.CSP.str.contains('is storm'))])
#print('accuracy:', (600-13-14-11)/600)

49

## DISCOURSE

In [623]:
# TSPW: 139 ok (when both cat and clear sky it goes for sky 77xPSTW); PSTW: 142 correct, when animal bird then always 75 WSPT; WSPT: should be 200 correct ; SPTW: 119 ok - 100 %;  no WPTS ok
# seems like it goes for the last solution...
# If [VAR:animal] is cat then the order is: T, S, P, W. If [VAR:sky] is clear then the order is: P, S, T, W. If [VAR:animal] is bird then the order is: W, S, P, T. Usually the order is: S, P, T, W.
dataset[~(dataset.Animal=='bird') & ~(dataset.Animal=='cat') & ~(dataset.Sky=='clear') & (dataset.S1.str.contains('S, P, T, W'))]
dataset[(dataset.S1.str.contains('W, P, T, S'))]

,ID,Sky,Temperature,Precipitation,Wind,Animal,Place,CSW,CSS,CSS_and,CSS_or,CSS_both,CSP,CSP_simple,CSP_know,CSP_gen,CSP_pizza,CSP_drizzle,CSP_shower,S1


In [741]:
# S2: 64 letters remained wind: 30 ; temperature 2 not translated ; sky 40; precipitation ok; 339 really correct
# dataset[~(dataset.S2.str.contains('rain') | dataset.S2.str.contains('snow') | dataset.S2.str.contains('storm') ) & ~(dataset.S2.str.contains('W') | dataset.S2.str.contains('P') | dataset.S2.str.contains('S'))]
dataset[(dataset.S2.str.contains('rain') | dataset.S2.str.contains('snow') | dataset.S2.str.contains('storm')) & dataset.S2.str.contains('temperature') & dataset.S2.str.contains('wind') & ((dataset.S2.str.contains('clear') | dataset.S2.str.contains('cloudy') | dataset.S2.str.contains('overcast') | dataset.S2.str.contains('smog') | dataset.S2.str.contains('mist')))]
# len(dataset[~dataset.S2.str.contains('temperature') & ~(dataset.S2.str.contains('W') | dataset.S2.str.contains('P') | dataset.S2.str.contains('S'))])
# dataset[~(dataset.S2.str.contains('clear') | dataset.S2.str.contains('cloudy') | dataset.S2.str.contains('overcast') | dataset.S2.str.contains('smog') | dataset.S2.str.contains('mist')) & ~(dataset.S2.str.contains('W') | dataset.S2.str.contains('P') | dataset.S2.str.contains('S'))]
print('accuracy:', (600-64-30-2-40)/600)

accuracy: 0.7733333333333333


In [748]:
pd.set_option('display.max_colwidth', None)
dataset[['S1', 'S2']].sample(30)

,S1,S2
87,"S, P, T, W",The sky is overcast. There is rain 4 mm/h. The temperature is 18 °C. There is strong wind 73 km/h.
101,"T, S, P, W",The temperature is 11 °C. There is smog. There is rain 18 mm/h. There is strong wind 53 km/h.
428,"P, S, T, W","P, S, T, W"
317,"T, S, P, W",The temperature is 14 °C. There is mist. There is no rain and no snow 0 mm/h. There is wind 43 km/h.
408,"T, S, P, W",The temperature is 22 °C. The sky is overcast. There is rain 12 mm/h. There is wind 33 km/h.
126,"P, S, T, W","P, S, T, W"
279,"W, S, P, T",There is strong wind 52 km/h. The sky is overcast. There is storm 7 mm/h. The temperature is -10 °C.
579,"T, S, P, W",There is mist. There is snow 19 mm/h. The temperature is -6 °C. There is strong wind 63 km/h.
91,"S, P, T, W",The sky is cloudy. There is storm. 18 mm/h. The temperature is 7 °C. There is strong wind 72 km/h.
154,"W, S, P, T",There is wind 48 km/h. The sky is clear. There is no rain and no snow 4 mm/h. The temperature is 4 °C.


In [771]:
# CSA 184x fish -> 109 instances removed, 10 wind kept, quite ok, interestingly it removes also "W" (sometimes of course was not there to remove... but whatever) BUT 211 others have wind removed!!! 176 ok
#If [VAR:animal] is fish then remove the sentence containing word "wind". If the [VAR:animal] is not fish, output all the provided text.
dataset[~(dataset.Animal=='fish') & ~(dataset.CSA.str.contains('wind'))]# & (dataset.S2.str.contains('wind'))]
# has wind ok: 176; has wind not ok 10; no wind ok 174; no wind not ok 240
# ok: 109+176; wrong=10+211 (506 that we can count for this)

,ID,Sky,Temperature,Precipitation,Wind,Animal,Place,CSW,CSS,CSS_and,CSS_or,CSS_both,CSP,CSP_simple,CSP_know,CSP_gen,CSP_pizza,CSP_drizzle,CSP_shower,S1,S2,CSA
0,0,overcast,6,4,23,bird,city,There is wind,There is smog,There is smog,There is smog,There is smog,There is rain,There is rain.,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is wind 23 km/h. There is smog. There is rain 4 mm/h. The temperature is 6 °C.,There is smog. There is rain 4 mm/h. The temperature is 6 °C.
5,5,overcast,5,0,54,bird,city,There is strong wind,There is smog,There is smog,There is smog,There is smog,There is no rain and no snow,There is no rain and no snow.,There is no rain and no snow.,There is no rain and no snow.,There is no rain and no snow.,There is no rain and no snow.,There is no rain and no snow.,"W, S, P, T",There is strong wind 54 km/h. There is smog. There is no rain and no snow 0 mm/h. The temperature is 5 °C.,There is smog. There is no rain and no snow 0 mm/h. The temperature is 5 °C.
7,7,overcast,28,9,24,cat,beach,There is wind,There is mist,There is mist,There is mist,There is mist,There is rain,There is rain.,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"T, S, P, W",The temperature is 28 °C. There is mist. There is rain 9 mm/h. There is wind 24 km/h.,The temperature is 28 °C. There is mist. There is rain 9 mm/h.
12,12,clear,2,0,48,bird,mountains,There is wind,The sky is clear,The sky is clear,The sky is clear,The sky is clear,There is no rain and no snow,There is no rain and no snow,There is no rain and no snow.,No snow,There is no rain and no snow.,There is no rain and no snow.,There is no rain and no snow.,"W, S, P, T",There is wind 48 km/h. The sky is clear. There is no rain and no snow 0 mm/h. The temperature is 2 °C.,There is no rain and no snow 0 mm/h. The temperature is 2 °C.
13,13,overcast,27,5,40,bird,city,There is wind,There is smog,There is smog,There is smog,There is smog,There is rain,There is rain.,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is wind 40 km/h. There is smog. There is rain 5 mm/h. The temperature is 27 °C.,There is smog. There is rain 5 mm/h. The temperature is 27 °C.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,583,cloudy,9,7,15,bird,beach,There is no wind,There is mist,There is mist,There is mist,There is mist,There is rain,There is rain.,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is no wind 15 km/h. There is mist. There is rain 7 mm/h. The temperature is 9 °C.,There is mist. There is rain 7 mm/h. The temperature is 9 °C.
588,588,overcast,10,11,75,bird,beach,There is strong wind.,There is mist,There is mist,There is mist,There is mist,There is rain,There is rain.,There is no rain and no snow.,There will be rain.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is strong wind 75 km/h. There is mist. There is rain 11 mm/h. The temperature is 10 °C.,There is mist. There is rain 11 mm/h. The temperature is 10 °C.
593,593,cloudy,28,16,34,bird,city,There is wind,There is smog,There is smog,There is smog,There is smog,There is rain,There is rain.,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is wind 34 km/h. There is smog. There is rain 16 mm/h. The temperature is 28 °C.,There is smog. There is rain 16 mm/h. The temperature is 28 °C.
596,596,cloudy,29,11,39,bird,mountains,There is wind,The sky is cloudy,The sky is cloudy,The sky is cloudy,The sky is cloudy,There is storm.,There is rain.,There is rain.,There will be rain and snow.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is wind 39 km/h. The sky is cloudy. There is storm. 11 mm/h. The temperature is 29 °C.,There is storm. 11 mm/h. The temperature is 29 °C.


## REALIZATION

In [780]:
pd.set_option('display.max_colwidth', None)
dataset[['S2', 'Falcon']].sample(30)

,S2,Falcon
236,The temperature is 5 °C. There is smog. There is rain 5 mm/h. There is wind 44 km/h.,"\nTomorrow in the city, the temperature will be 5 °C. There will be smog. There will be 5 mm/h of rain. There will be a gentle breeze blowing at 44 km/h."
132,The temperature is 15 °C. The sky is overcast. There is storm. 5 mm/h. There is no wind 2 km/h.,"\nThe weather forecast for tomorrow is for the mountains. The temperature will be 15 °C, and there will be a storm with 5 mm/h of precipitation. There will be no wind at 2 km/h."
186,There is no wind 13 km/h. The sky is clear. There is no rain and no snow 4 mm/h. The temperature is -7 °C.,"\nTomorrow on the meadows, the weather will be calm with no wind at 13 km/h. The sky will be clear, and there will be no rain or snow. The temperature will be -7 °C."
573,"P, S, T, W","'Tomorrow, on the meadows, P, S, T, W.'"
425,There is wind 47 km/h. The sky is cloudy. There is storm 8 mm/h. The temperature is 25 °C.,"\nTomorrow on the mountains, there will be a strong wind at 47 km/h. The sky will be cloudy, and there will be a storm with 8 mm/h. The temperature will be 25 °C."
16,The sky is overcast. There is no rain and no snow 0 mm/h. The temperature is 4 °C. There is wind 27 km/h.,"\nThe weather forecast for tomorrow is that the mountains will be overcast with no rain or snow. The temperature will be 4°C, and there will be a gentle breeze blowing at 27 km/h."
94,There is wind 23 km/h. The sky is cloudy. There is no rain and no snow 0 mm/h. The temperature is 20 °C.,"\nTomorrow on the mountains, the weather will be windy with a speed of 23 km/h. The sky will be cloudy, and there will be no rain or snow. The temperature will be 20 °C."
338,The temperature is 3 °C. The sky is overcast. There is rain 8 mm/h. There is strong wind 77 km/h.,\nThe weather forecast for tomorrow is: Expect a mild day with temperatures reaching 3 °C on the meadows. Cloudy skies with 8 mm/h of rain will likely dominate the day. A strong wind with 77 km/h will make it feel cooler than the actual temperature.
191,The temperature is -1 °C. There is smog. There is snow 5 mm/h. There is wind 19 km/h.,"\nTomorrow in the city, the temperature will be -1 °C. There will be smog. There will be 5 mm/h of snow. There will be a strong wind at 19 km/h."
212,There is no rain and no snow 0 mm/h. The temperature is 12 °C. There is strong wind 54 km/h.,"\nTomorrow on the beach, there will be no rain or snow, and the temperature will be a comfortable 12 °C. A strong wind will blow at 54 km/h, making it a great day to enjoy the beach."
